**Notebook: Datos**

<img src="http://oci02.img.iteso.mx/identidad_de_instancia_2018/ITESO/Logos%20ITESO/Logo-ITESO-Principal.jpg">

<center> <font color= #004A94> <font size = 6> Msc Ciencia de datos </font> <br> <br> <font color= #047CFB> <font size = 4>I.F. Juan Francisco Muñoz Elguezabal - franciscome@iteso.mx </font>

<center><font color= #004A94> <font size = 6> ANÁLISIS ESTADÍSTICO MULTIVARIABLE </font> <br> <br>

<center><font color=#42c29b><font size=5> Una comparación entre modelos regresivos lineales y clustering secuencial como predictores de series de tiempo financieras </font> <br> <br>

<center><font color=#047CFB><font size=5> El caso  de una estrategia de trading para el mercado internacional de divisas </font> <br> <br> 
    
<center> <font color= #047CFB> <font size = 4> Repositorio: <a href='https://github.com/IFFranciscoME/FinTechLab/tree/master/'>Link</a></font>

---

## <span style="color:#004A94"> Cargar librerías </span>

In [6]:
import numpy as np                                        # funciones numericas
import pandas as pd                                       # dataframes y utilidades
from datetime import timedelta                            # diferencia entre datos tipo tiempo
from oandapyV20 import API                                # conexion con broker para informacion historica
import oandapyV20.endpoints.instruments as instruments    # informacion de precios historicos

## <span style="color:#004A94"> Leer archivos </span>

In [4]:
# -- --------------------------------------------------------------------------- Solo cargar datos pre-descargados -- #
# leer archivo de precios historicos ya descargados previamente
df_pe_m5 = pd.read_csv("archivos/Eur_Usd_M5.csv")
df_pe_w = pd.read_csv("archivos/Eur_Usd_W.csv")
df_ce = pd.read_csv("archivos/Calendario_Economico.csv")

## <span style="color:#004A94"> Generación de datos de calendario económico </span>

In [7]:
# seleccionar indicadores que hayan sido publicados 4 veces en cada mes (semanales)
df_ce = df_ce.iloc[np.where(df_ce['timestamp'] == '01/04/2016 19:30:00')[0][0]:len(df_ce['timestamp'])]
df_ce = df_ce.reset_index(drop=True)
df_ce['mes'] = [pd.to_datetime(df_ce['timestamp'][i]).strftime('%m') for i in range(0, len(df_ce['timestamp']))]
unicos = list(set(df_ce['Name']))
longitudes = [len(df_ce[df_ce['Name'] == unicos[i]]) for i in range(0, len(unicos))]
res = [idx for idx, val in enumerate(longitudes) if val > 12*3.5*4]
ind_semanales = [unicos[i] for i in res]

# funcion wow
df_ce_w = df_ce[df_ce['Name'].isin(ind_semanales)]
df_ce_w = df_ce_w.reset_index(drop=True)

In [13]:
df_ce.head()

,timestamp,Currency,Name,Country,Volatility,actual,previous,consensus,mes
0,01/04/2016 19:30:00,USD,CFTC USD NC Net Positions,United States,1,-95.2,NaN,NaN,01
1,01/04/2016 19:30:00,USD,CFTC Gold NC Net Positions,United States,1,19.1,NaN,NaN,01
2,01/04/2016 19:30:00,USD,CFTC Oil NC Net Positions,United States,1,196.4,NaN,NaN,01
3,01/04/2016 19:30:00,EUR,CFTC EUR NC Net Positions,European Monetary Union,1,-160.6,NaN,NaN,01
4,01/05/2016 08:55:00,EUR,Unemployment Change,Germany,2,-14.0,-13.0,NaN,01


In [12]:
ind_semanales

['API Weekly Crude Oil Stock',
 'CFTC EUR NC Net Positions',
 'CFTC USD NC Net Positions',
 'Consumer Price Index (MoM)',
 'CFTC Gold NC Net Positions',
 'Redbook Index (MoM)',
 'EIA Natural Gas Storage Change',
 'Baker Hughes US Oil Rig Count',
 'MBA Mortgage Applications',
 'Initial Jobless Claims',
 'Markit Services PMI',
 'CFTC Oil NC Net Positions',
 'Markit PMI Composite',
 'Consumer Price Index (YoY)',
 '4-Week Bill Auction',
 '3-Month Bill Auction',
 'Continuing Jobless Claims',
 'EIA Crude Oil Stocks Change',
 'Redbook Index (YoY)',
 '6-Month Bill Auction',
 'Markit Manufacturing PMI',
 'Consumer Confidence']

In [9]:
df_ce_w.head(10)

,timestamp,Currency,Name,Country,Volatility,actual,previous,consensus,mes
0,01/04/2016 19:30:00,USD,CFTC USD NC Net Positions,United States,1,-95.2,NaN,NaN,01
1,01/04/2016 19:30:00,USD,CFTC Gold NC Net Positions,United States,1,19.1,NaN,NaN,01
2,01/04/2016 19:30:00,USD,CFTC Oil NC Net Positions,United States,1,196.4,NaN,NaN,01
3,01/04/2016 19:30:00,EUR,CFTC EUR NC Net Positions,European Monetary Union,1,-160.6,NaN,NaN,01
4,01/05/2016 10:00:00,EUR,Consumer Price Index (YoY),European Monetary Union,3,0.2,0.20,0.3,01
5,01/05/2016 13:55:00,USD,Redbook Index (YoY),United States,1,2.9,2.50,NaN,01
6,01/05/2016 13:55:00,USD,Redbook Index (MoM),United States,1,-0.3,-0.50,NaN,01
7,01/05/2016 16:30:00,USD,4-Week Bill Auction,United States,1,0.2,0.17,NaN,01
8,01/05/2016 21:30:00,USD,API Weekly Crude Oil Stock,United States,1,-5.6,2.90,NaN,01
9,01/06/2016 07:45:00,EUR,Consumer Confidence,France,1,96.0,96.00,95.0,01


## <span style="color:#004A94"> Parámetros para petición de precios con OANDA </span>

In [5]:
# -- ------------------------------------------------------------------------------ Para descargar todos los datos -- #
# Token para API de OANDA
OA_Ak = '7' + '9ae0a52f8e483facdd81f5b316a8ef8-99fb5554f4739c76535b209044f7de2' + '6'
OA_In = "EUR_USD"                  # Instrumento
OA_Gn = "M5"                        # Granularidad de velas
fini = pd.to_datetime("2009-01-06 00:00:00").tz_localize('GMT')  # Fecha inicial
ffin = pd.to_datetime("2019-12-06 00:00:00").tz_localize('GMT')  # Fecha final

pd.set_option('display.max_rows', None)                   # sin limite de renglones maximos para mostrar pandas
pd.set_option('display.max_columns', None)                # sin limite de columnas maximas para mostrar pandas
pd.set_option('display.width', None)                      # sin limite el ancho del display
pd.set_option('display.expand_frame_repr', False)         # visualizar todas las columnas de un dataframe
pd.options.mode.chained_assignment = None                 # para evitar el warning enfadoso de indexacion

## <span style="color:#004A94"> Función para descarga masiva de precios con OANDA </span>

In [ ]:
# -- -------------------------------------------------------------------- FUNCION: Obtencion de precios historicos -- #
# -- -------------------------------------------------------------------- ---------------------------------------- -- #

def f_precios_masivos(p0_fini, p1_ffin, p2_gran, p3_inst, p4_oatk, p5_ginc):
    """
    :param p0_fini: str : fecha inicial para pedir precios : "2009-01-06 00:00:00"
    :param p1_ffin: str : fecha final para pedir precios : "2019-12-01 00:00:00"
    :param p2_gran: str : granularidad para pedir precios : S5, S10, S30, M1, M5, M15, M30, H1, H4, H8, D, W, M
    :param p3_inst: str : instrumento sobre el cual peedir precios (Acorde a nomenclatura OANDA) : EUR_USD
    :param p4_oatk: str : llave privada de oanda para peticiones con api : alfanumerica
    :param p5_ginc: int : numero limite de datos historicos a pedir segun Oanda : 5000

    :return: r_df_final: pd.DataFrame : dataframe final con todos los precios historicos
    """

    def f_datetime_range_fx(start_datetime, end_datetime, increment, delta_period):
        """
        :param start_datetime:
        :param end_datetime:
        :param increment:
        :param delta_period:

        :return:
        """

        result = []
        nxt = start_datetime

        while nxt <= end_datetime:
            result.append(nxt)
            if delta_period == 'minutes':
                nxt += timedelta(minutes=increment)
            elif delta_period == 'hours':
                nxt += timedelta(hours=increment)
            elif delta_period == 'days':
                nxt += timedelta(days=increment)

        return result

    # inicializar api de OANDA
    api = API(access_token=p4_oatk)

    gn = {'S30': 30, 'S10': 10, 'S5': 5, 'M1': 60, 'M5': 60*5, 'M15': 60*15, 'M30': 60*30,
          'H1': 60*60, 'H4': 60*60*4, 'H8': 60*60*8, 'D': 60*60*24, 'W': 60*60*24*7, 'M': 60*60*24*7*4}

    # -- para el caso donde con 1 peticion se cubran las 2 fechas
    if int((p1_ffin - p0_fini).total_seconds() / gn[p2_gran]) < 5000:

        # Fecha inicial y fecha final
        f1 = p0_fini.strftime('%Y-%m-%dT%H:%M:%S')
        f2 = p1_ffin.strftime('%Y-%m-%dT%H:%M:%S')

        # Parametros pra la peticion de precios
        params = {"granularity": p2_gran, "price": "M", "dailyAlignment": 16, "from": f1, "to": f2}

        # Ejecutar la peticion de precios
        a1_req1 = instruments.InstrumentsCandles(instrument=p3_inst, params=params)
        a1_hist = api.request(a1_req1)

        # Para debuging
        # print(f1 + ' y ' + f2)
        lista = list()

        # Acomodar las llaves
        for i in range(len(a1_hist['candles']) - 1):
            lista.append({'TimeStamp': a1_hist['candles'][i]['time'],
                          'Open': a1_hist['candles'][i]['mid']['o'],
                          'High': a1_hist['candles'][i]['mid']['h'],
                          'Low': a1_hist['candles'][i]['mid']['l'],
                          'Close': a1_hist['candles'][i]['mid']['c']})

        # Acomodar en un data frame
        r_df_final = pd.DataFrame(lista)
        r_df_final = r_df_final[['TimeStamp', 'Open', 'High', 'Low', 'Close']]
        r_df_final['TimeStamp'] = pd.to_datetime(r_df_final['TimeStamp'])

        return r_df_final

    # -- para el caso donde se construyen fechas secuenciales
    else:

        # hacer series de fechas e iteraciones para pedir todos los precios
        fechas = f_datetime_range_fx(start_datetime=p0_fini, end_datetime=p1_ffin, increment=p5_ginc,
                                     delta_period='minutes')

        # Lista para ir guardando los data frames
        lista_df = list()

        for n_fecha in range(0, len(fechas)-1):

            # Fecha inicial y fecha final
            f1 = fechas[n_fecha].strftime('%Y-%m-%dT%H:%M:%S')
            f2 = fechas[n_fecha+1].strftime('%Y-%m-%dT%H:%M:%S')

            # Parametros pra la peticion de precios
            params = {"granularity": p2_gran, "price": "M", "dailyAlignment": 16, "from": f1, "to": f2}

            # Ejecutar la peticion de precios
            a1_req1 = instruments.InstrumentsCandles(instrument=p3_inst, params=params)
            a1_hist = api.request(a1_req1)

            # Para debuging
            print(f1 + ' y ' + f2)
            lista = list()

            # Acomodar las llaves
            for i in range(len(a1_hist['candles']) - 1):
                lista.append({'TimeStamp': a1_hist['candles'][i]['time'],
                              'Open': a1_hist['candles'][i]['mid']['o'],
                              'High': a1_hist['candles'][i]['mid']['h'],
                              'Low': a1_hist['candles'][i]['mid']['l'],
                              'Close': a1_hist['candles'][i]['mid']['c']})

            # Acomodar en un data frame
            pd_hist = pd.DataFrame(lista)
            pd_hist = pd_hist[['TimeStamp', 'Open', 'High', 'Low', 'Close']]
            pd_hist['TimeStamp'] = pd.to_datetime(pd_hist['TimeStamp'])

            # Ir guardando resultados en una lista
            lista_df.append(pd_hist)

        # Concatenar todas las listas
        r_df_final = pd.concat([lista_df[i] for i in range(0, len(lista_df))])

        columns = list(r_df_final.columns)
        columns = [i.lower() for i in columns]

        # Renombrar todas las columnas del dataframe con una lista de nombres
        r_df_final.rename(columns=dict(zip(r_df_final.columns[0:], columns)), inplace=True)

        # resetear index en dataframe resultante porque guarda los indices del dataframe pasado
        r_df_final = r_df_final.reset_index(drop=True)

        return r_df_final


print('Fin proceso cargar datos')

## <span style="color:#004A94"> Descarga masiva de precios y creación de archivo </span>

In [ ]:

# -- -------------------------------------------------------------------------------- Proceso de descarga completo -- #
# -- -------------------------------------------------------------------------------- ---------------------------- -- #
# -- Solo correr esta parte si se quiere descargar todos los precios

# Descagar todos los precios necesarios (descomentar la linea de import funciones as fn)
# df_pe = f_precios_masivos(p0_fini=fini, p1_ffin=ffin, p2_gran=OA_Gn, p3_inst=OA_In, p4_oatk=OA_Ak, p5_ginc=5000)

# Escribir dataframe en un archivo csv
# df_pe.to_csv(r"archivos/Eur_Usd_M5.csv", index=False)
